<a href="https://colab.research.google.com/github/mibeaum/ML/blob/master/TimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Time series classification**



Load Python modules

In [4]:
import numpy as np  # Arrays, matrices and functions on them. Required by Pandas, below
import pandas as pd # A data analysis library
from sklearn.model_selection import train_test_split # scikit-learn, machine learning tools
import matplotlib.pyplot as plt # A plotting library
import seaborn as sns # Built on matplotlib, facilitates aesthetically pleasing plots

# General settings
sns.set_style('whitegrid') # Plots will have a white grid
# Variables that will help us work with the classes
class_names = ['cement', 'carpet']
class_colors = ['darkorange', 'steelblue']

Functions

In [39]:
def load_data(filename):
    ''' Load the data from a file in a GitHub repo '''
    url_root = 'https://raw.githubusercontent.com/mibeaum/ML/master'
    url = url_root+'/'+filename
    df = pd.read_csv(url, sep=',', header=0, usecols = ["server", "dateTime", "internalPacketIdentifier"]) # Use Pandas to load the data into a Pandas DataFrame
    print('Loaded from', url)
    data = df.values # Convert from a Pandas DataFrame to a numpy array
    print('The shape of data is', data.shape)
    print('')
    return data

def plot_data_samples(data, labels, sample_numbers):
    ''' Plot the time series data relating to the input list of sample numbers '''
    # Input format - a list, e.g. [1, 7, 22, 42]
    fig, ax = plt.subplots()

    for i in sample_numbers:
        plt.plot(data[i], label=class_names[labels[i]], color=class_colors[labels[i]])
        print('sample', i, 'class', str(labels[i]), class_names[labels[i]])

    print('')
    plt.ylim([-3.5, 3.5])
    plt.title('Orange : cement (class 0)\nBlue : carpet (class 1)')
    ax.set_ylabel('Accelerometer data')
    ax.set_xlabel('Data point number')    

Load the data

In [40]:
filename = 'data.csv'
data_log = load_data(filename) 
print('The data is a matrix. These are the first 7 rows and 3 columns of data:\n', data_log[:7, :3], '\n')

Loaded from https://raw.githubusercontent.com/mibeaum/ML/master/data.csv
The shape of data is (42, 3)

The data is a matrix. These are the first 7 rows and 3 columns of data:
 [[0 '01/06/202014:49:12' '0000026335FA1560']
 [0 '01/06/202014:49:12' '0000026335FA1560']
 [1 '01/06/202019:49:12' '0000026335C83DB0']
 [1 '01/06/202019:49:12' '0000026335C83DB0']
 [0 '01/06/202014:49:12' '0000026338FC9920']
 [0 '01/06/202014:49:12' '0000026338FC9920']
 [0 '01/06/202014:49:12' '00000263351894F0']] 



Process the data

In [41]:
labels = data_log[:,0].astype(int)
data = data_log[:,1:]
print('The shape of the data matrix is', data.shape)
print('The shape of the labels vector is', labels.shape)

The shape of the data matrix is (42, 2)
The shape of the labels vector is (42,)


Plot the data